<a href="https://colab.research.google.com/github/sadamzuoby/Financial-analysis-tools/blob/main/Financial_and_accounting_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

التحليل المحاسبي

In [ ]:
!pip install PyMuPDF
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 29.4 MB/s eta 0:00:00


**رفع** ملفات PDF للقوائم المالية

In [ ]:
from google.colab import files

uploaded = files.upload()


 استخراج النص من ملفات PDF باستخدام PyMuPDF

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text


دالة لاستخراج القيم من النص باستخدام الكلمات المفتاحية

In [ ]:
def extract_financial_value(text, keyword):
    if isinstance(keyword, list):
        for k in keyword:
            value = extract_financial_value(text, k)
            if value is not None:
                return value
        return None

    # السماح بمسافة أو فاصل قبل الرقم
    pattern = rf"{keyword}[^0-9\-–]*([\d.,]+)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        number = match.group(1).replace(',', '').replace('٬', '').replace('٫', '.')
        try:
            return float(number)
        except:
            return None
    return None


تجهيز البيانات وتحليل نسبة التغيير

In [ ]:
# الكلمات المفتاحية لكل بند
actual_keywords = {
    "إجمالي الموجودات": ["مجموع الموجودات", "إجمالي الموجودات"],
    "إجمالي التمويلات": [
        "التمويلات – بالصافي كما وردت في قائمة المركز المالي الموحدة كما في 31 كانون الأول",
        "إجمالي أرصدة التمويل والاستثمار",
        "أرصدة التمويل – صافي",
        "إجمالي التوظيفات",
        "إجمالي التمويلات"
    ],
    "حقوق الملكية": [
        "حقوق الملكية",
        "إجمالي حقوق الملكية",
        "مجموع حقوق الملكية",
        "مجموع حقوق الملكية – مساهمي البنك"
    ],
    "صافي الأرباح": [
        "الربح بعد الضريبة",
        "صافي الربح",
        "صافي الأرباح"
    ],
    "الإيرادات التشغيلية": [
        "إجمالي الدخل",
        "إجمالي الإيرادات",
        "الإيرادات التشغيلية"
    ]
}

# دالة لاستخراج القيم أو طلبها يدويًا
def extract_or_prompt_value(text_lines, keywords_dict):
    results = {}
    for key, keyword_list in keywords_dict.items():
        found = False
        for line in text_lines:
            for keyword in keyword_list:
                if keyword in line:
                    # محاولة استخراج القيمة الرقمية من السطر
                    import re
                    numbers = re.findall(r'[\d,\.]+', line)
                    if numbers:
                        try:
                            value = float(numbers[0].replace(',', ''))
                            results[key] = value
                            found = True
                            break
                        except ValueError:
                            continue
            if found:
                break
        if not found:
            # طلب إدخال يدوي عند الفشل في العثور
            while True:
                manual_value = input(f"لم يتم العثور على '{key}' تلقائيًا. الرجاء إدخال القيمة يدويًا: ")
                try:
                    results[key] = float(manual_value.replace(',', ''))
                    break
                except ValueError:
                    print("⚠️ الرجاء إدخال رقم صالح (مثال: 123456.78 أو 123,456).")
    return results

# مثال تجريبي: سطور نصية مأخوذة من PDF (أو مفترضة)
text_from_pdf = [
    "إجمالي الموجودات 3,000,000",
    "حقوق الملكية 1,200,000",
    "الربح بعد الضريبة 350,000"
    # البقية غائبة لاختبار الإدخال اليدوي
]

# تشغيل الدالة
results = extract_or_prompt_value(text_from_pdf, actual_keywords)
print("النتائج النهائية:")
for item, value in results.items():
    print(f"{item}: {value:,.2f}")


لم يتم العثور على 'إجمالي التمويلات' تلقائيًا. الرجاء إدخال القيمة يدويًا: 36,826,085
لم يتم العثور على 'الإيرادات التشغيلية' تلقائيًا. الرجاء إدخال القيمة يدويًا: 33,157,539
النتائج النهائية:
إجمالي الموجودات: 3,000,000.00
إجمالي التمويلات: 36,826,085.00
حقوق الملكية: 1,200,000.00
صافي الأرباح: 350,000.00
الإيرادات التشغيلية: 33,157,539.00


حساب نسبة التغير لكل بند

In [ ]:
def calculate_change(values):
    changes = []
    for i in range(1, len(values)):
        if values[i-1] is not None and values[i] is not None:
            change = ((values[i] - values[i-1]) / values[i-1]) * 100
            changes.append(round(change, 2))
        else:
            changes.append(None)
    return changes


عرض النتائج

In [ ]:
def calculate_change(values_dict):
    # ترتيب السنوات تصاعديًا
    sorted_years = sorted(values_dict.keys())

    # استخراج القيم بحسب ترتيب السنوات
    values = [values_dict.get(year) for year in sorted_years]

    # حساب نسبة التغير
    changes = []
    for i in range(1, len(values)):
        prev = values[i - 1]
        curr = values[i]
        if prev and curr and prev != 0:
            change = ((curr - prev) / prev) * 100
            changes.append(round(change, 2))
        else:
            changes.append(None)

    return sorted_years, values, changes



In [ ]:
from tabulate import tabulate

table = []

# المرور على كل بند
for item, yearly_values in financial_data.items():
    years, values, changes = calculate_change(yearly_values)

    # إنشاء صف: [البند, 2021 القيمة, 2022 القيمة, ..., % تغير 1→2, % تغير 2→3, ...]
    row = [item]
    row += [f"{v:,.2f}" if v is not None else "N/A" for v in values]
    row += [f"{c:.2f}%" if c is not None else "N/A" for c in changes]
    table.append(row)

# إنشاء رؤوس الأعمدة
headers = ["البند"]
headers += years
headers += [f"نسبة التغير ({years[i-1]}→{years[i]})" for i in range(1, len(years))]

# عرض الجدول
print(tabulate(table, headers=headers, tablefmt="grid", numalign="right", stralign="center"))



+---------------------+----------------+----------------+----------+---------------------------+---------------------------+
|        البند        |      2021      |      2022      |   2023   |  نسبة التغير (2021→2022)  |  نسبة التغير (2022→2023)  |
+=====================+================+================+==========+===========================+===========================+
|  إجمالي الموجودات   |      9.60      |      9.50      |   9.50   |          -1.04%           |           0.00%           |
+---------------------+----------------+----------------+----------+---------------------------+---------------------------+
|  إجمالي التمويلات   |      N/A       |      N/A       |   N/A    |            N/A            |            N/A            |
+---------------------+----------------+----------------+----------+---------------------------+---------------------------+
|    حقوق الملكية     |     230.00     |      1.00      |  256.00  |          -99.57%          |         25500.00%         |
